# Re-sampling

the code bellow is dedicated to present the ressampling of the image

- As is presented bellow the calculos of the b11 needed to be reshaped as we needed to encrease the matrix by two so the file encreased by the scale of two. 

- WE only needed the reescale the frame at band 11 to try the best not loss information thats why we didnt decrease the spatial resolution at the other bands for exemple this band 11 of sentinel 2 represents short wave infrared around the biggning of the middle infrared band around 1.6 micrometers 

02/04/2022 and 02/05/2022 were the exemples used 

In [18]:
import rasterio
from rasterio.enums import Resampling


file = '/home/queimadas/Documents/INPE/bdgeo/data/S2A_MSIL1C_20220402T133231_N0400_R081_T22LHH_20220402T171237.SAFE/GRANULE/L1C_T22LHH_A035399_20220402T133450/IMG_DATA/T22LHH_20220402T133231_'

input_image = file + 'B11.jp2'
output_image = file + 'B11_Resampling.jp2'

with rasterio.open(input_image) as src:

    scale_factor = 2  
    new_width = src.width * scale_factor
    new_height = src.height * scale_factor
    new_transform = src.transform * src.transform.scale(
        (src.width / new_width),
        (src.height / new_height)
    )    
    
    data = src.read(
        out_shape=(src.count, int(new_height), int(new_width)),
        resampling=Resampling.bilinear
    )
    
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": new_height,
        "width": new_width,
        "transform": new_transform
    })
    
    
    with rasterio.open(output_image, "w", **out_meta) as dest:
        dest.write(data)


# Creating the Index 

For the usage of different images we do need to use the bands at the same resolution for this reason we tried the most to use the same resolution as in the satellites from sentinel 2 constellation there are multiple resolutions. For our exemple we did use the ndvi and prsi at 10 meters while the ndii and nbr resolution are 20 meters. This was used with the idea of using less ressample as possible. 

In [ ]:

import rasterio
import numpy as np


def calculate_indices(red, nir1,nir2, swir1,swir1r, swir2, blue):

    ndvi = (nir1- red) / (nir1 + red + epsilon)
    
    nbr = (nir2 - swir2) / (nir2 + swir2 + epsilon)

    ndii = (nir2 - swir1) / (nir2 + swir1+ epsilon)

    prsi = (red - blue) / (swir1r + epsilon)
    
    return ndvi, nbr, ndii, prsi

arquivo = '/home/queimadas/Documents/INPE/bdgeo/data/S2A_MSIL1C_20220402T133231_N0400_R081_T22LHH_20220402T171237.SAFE/GRANULE/L1C_T22LHH_A035399_20220402T133450/IMG_DATA/T22LHH_20220402T133231_'
red_band_path = arquivo+'B04.jp2'
nir_band_path_10m = arquivo+'B08.jp2'
nir_band_path = arquivo+'B8A.jp2'
swir1_band_path_r = arquivo + 'B11_Resampling.jp2'
swir1_band_path = arquivo + 'B11.jp2'
swir2_band_path = arquivo + 'B12.jp2'
blue_band_path = arquivo + 'B02.jp2'

with rasterio.open(red_band_path) as red_src:
    red = red_src.read(1).astype('float32')

with rasterio.open(nir_band_path_10m) as nir_src:
    nir1 = nir_src.read(1).astype('float32')
    
with rasterio.open(nir_band_path) as nir_src:
    nir = nir_src.read(1).astype('float32')
    
with rasterio.open(swir1_band_path) as swir1_src:
    swir1 = swir1_src.read(1).astype('float32')

with rasterio.open(swir1_band_path_r) as swir1_src:
    swir1_r = swir1_src.read(1).astype('float32')
    

with rasterio.open(swir2_band_path) as swir2_src:
    swir2 = swir2_src.read(1).astype('float32')
    
with rasterio.open(blue_band_path) as blue_src:
    blue = blue_src.read(1).astype('float32')

epsilon = 1e-10

ndvi, nbr, ndii, prsi = calculate_indices(red,nir1, nir, swir1,swir1_r, swir2, blue)

ndvi_output_path = 'path_to_save_ndvi_old.tif'
nbr_output_path = 'path_to_save_nbr_old.tif'
ndii_output_path = 'path_to_save_ndii_old.tif'
prsi_output_path = 'path_to_save_prsi_old.tif'

def save_raster(output_path, data, reference_raster):
    profile = reference_raster.profile
    profile.update(
        driver='GTiff', 
        dtype=rasterio.float32, 
        count=1
    )
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(data, 1)

with rasterio.open(red_band_path) as ref_raster:
    save_raster(ndvi_output_path, ndvi, ref_raster)
    save_raster(nbr_output_path, nbr, ref_raster)
    save_raster(ndii_output_path, ndii, ref_raster)
    save_raster(prsi_output_path, prsi, ref_raster)

print("Indices calculated and saved successfully.")